# FIML - IV


Resolvendo o FIML como se fosse IV, então temos que $det(\Gamma) = 1$. Dessa forma, basta resolver $\min |U'U|$

In [1]:


using LinearAlgebra
using Symbolics
using MultivariatePolynomials
using DynamicPolynomials
using HomotopyContinuation

# função para arredondar polinomios 
function roundp(p)
      if p isa AbstractPolynomialLike
          cs = coefficients(p)
          ms = monomials(p)
          return isempty(cs) ? zero(p) : sum(round(c; digits=1) * m for (c, m) in zip(cs, ms))
      elseif p isa Number
          return round(p; digits=2)
      elseif p isa AbstractArray
          return map(roundp, p)
      else
          error("Unsupported type in roundp: $(typeof(p))")
      end
  end
  # --------------------
  


# Definição das variáveis
n = 100
k = 3
p = 2
q = p-1

m1 = p * p
m2 = k * p

X = rand(n, k)
Y = rand(n, p)

R1 = [ ones(1, 1)                 zeros(1, q)                         zeros(1, p*q);
       zeros(q, 1)              zeros(q, q)                       zeros(q, p*q);
       zeros(p*q, 1)          zeros(p*q, q)                   Matrix( I, p*q, p*q )  ]

R2 = [ Matrix(I, k, k)                 zeros(k, k*q);
       zeros(k*q, k)               zeros(k*q, k*q)  ]

 
display("oi")

"oi"

In [ ]:
# --- Crie variáveis simbólicas (tipo Num), não Symbols ---

@polyvar γ[1:q] b[1:k*q]

Γ = [ 1                    zeros(1, q);
      γ                    Matrix(I, q, q) ]


B = [ zeros(k, 1)   reshape(b, k, q) ]        

U = Y * Γ - X * B        # n×p (simbólico)

# --- vetores lambda


@polyvar λ[ 1: p^2 - q ]   μ[1:k]

lambda1 = [ λ[1] ; zeros(q,1) ; λ[2:end]  ]   # m1×1
lambda2 = [  μ[1:k] ; zeros(m2 - k , 1) ]   # m2×1



In [ ]:
display(B)
display(Γ)

In [ ]:
# equações

eqA = reshape(Y' * U, p*p, 1) +
      kron(U' * U, Matrix(I, p, p)) * R1 * lambda1

eqB = reshape((X' * U)', k*p, 1) +
      kron(U' * U, Matrix(I, k, k)) * R2 * lambda2



# final

F = vec( [eqA ; eqB] ) 

vars = [ γ ; b ; λ ; μ  ]


F_sys = System(F; variables=vars)

#result = solve(F_sys)
result = solve(F_sys ; start_system = :total_degree)

rounded_solutions = [ real.(round.(sol; digits=4)) for sol in solutions(result)  ]


In [ ]:
vars

# Tem que terminar aqui pra baixo

In [ ]:
resultados_sol = []
for sol in solutions(result) 
    Aux = real.(round.(sol[1:6]; digits=2))

Γ_eu = [ 1                         zeros(1, q);
         Aux[1]                    Matrix(I, q, q) ]


B_eu = [ zeros(k, 1)   reshape(b, k, q) ]  
    
    val = VpV(Y, X, Γ_eu, B_eu)
    push!(resultados_sol, (Γ = Γ_eu, Bs = B_eu, detVpV=val))
    display(val)
end


min_val, pos = findmin(r[:detVpV] for r in resultados_sol)

println("Mínimo = $min_val na posição $pos")

# agora pega a tupla correspondente
Resultado1 = resultados_sol[pos]

println("Par ótimo:")
println("Γ = ", Resultado1.Γ)
println("Bs = ", Resultado1.Bs)

## Segunda forma de resolver



In [2]:
@var b[1:k*q] gam[1:q]

Γ = [ 1                      zeros(1, q);
      gam                    Matrix(I, q, q) ]


B = [ zeros(k, 1)   reshape(b, k, q) ]  


3×2 Matrix{Expression}:
 0.0  b₁
 0.0  b₂
 0.0  b₃

In [3]:
U = (Symbolics.expand.(simplify.(Y*Γ - X*B)))

UpU = U'U

detU = det(UpU)


eqa1 = R1* kron( detU*inv(UpU), Y')*vec(U)  
eqa2 = R2* kron( detU*inv(UpU), X')*vec(U)

eqf_1 = (simplify.(eqa1))
eqf_2 = (simplify.(eqa2))


F = vec( [eqf_1 ; eqf_2] ) 
vars = [ gam ; b ; lam1 ; lam2  ]
F_sys = System(F; variables=vars)

display("oi")

LoadError: ArgumentError: matrix contains Infs or NaNs

In [ ]:
result = solve(F_sys)
#result = solve(F_sys ; start_system = :total_degree)

rounded_solutions = [ real.(round.(sol; digits=4)) for sol in solutions(result)  ]


# Caso queiramos ver U'U



In [ ]:
function UpU(gamma1, beta)
      B_1 = [ zeros(k, 1)   beta ] 
      Γ_1 = [ 1               zeros(1, q);
              gamma1          Matrix(I, q, q) ]
    
      U = Y*Γ_1 - X*B_1 
    
      return det(U'*U)
end

In [ ]:
xs = range(-5, 5; length=2000)
ys = range(-5, 5; length=2000)

#Z = [VpV(x, y) for x in xs, y in ys]

Z_2 = [UpU(x, y) for x in xs, y in ys]

display("oi")

In [ ]:
#display(minimum(Z))
display(minimum(Z_2))

In [ ]:
#display(VpV(-0.6,0.89))

#display(VpV(-4.81, 0))

In [ ]:
display(UpU(-0.6,0.89))

display(UpU(-4.81, 0))

# Curvas de nível

In [ ]:
using Plots

contour(xs, ys, Z_2;
        levels=20,
        xlabel="x", ylabel="y",
        legend=false, aspect_ratio=1, framestyle=:box)

x0, y0 = -0.60, 0.89

scatter!([x0], [y0];
    markershape=:circle,
    markersize=3,                 # bem maior
    color=:red,
    markerstrokecolor=:black,      # contorno preto
    markerstrokewidth=2,
    label="mínimo")


x1, y1 = -4.81, 0

scatter!([x1], [y1];
    markershape=:circle,
    markersize=3,                 # bem maior
    color=:red,
    markerstrokecolor=:black,      # contorno preto
    markerstrokewidth=2,
    label="mínimo")


In [ ]:
contour(xs, ys, Z;
        levels=20,
        xlabel="x", ylabel="y",
        legend=false, aspect_ratio=1, framestyle=:box)


scatter!([x0], [y0];
    markershape=:circle,
    markersize=3,                 # bem maior
    color=:red,
    markerstrokecolor=:black,      # contorno preto
    markerstrokewidth=2,
    label="mínimo")

scatter!([x1], [y1];
    markershape=:circle,
    markersize=3,                 # bem maior
    color=:red,
    markerstrokecolor=:black,      # contorno preto
    markerstrokewidth=2,
    label="mínimo")


# LIML

In [ ]:

# Conferir com LIML 


M = Matrix(I , n , n) - X * inv(X' * X) * X'


function Q(x)
      numerador = x' * Y' * Y * x
      denominador = x' * Y' * M * Y * x
      return numerador / denominador
end


λ, V = eigen(Y' * Y, Y' * M * Y)

V

V_normalizada = hcat([V[:,j] / V[1,j] for j in 1:size(V,2)]...)


V_normalizada



In [ ]:
Q(  [1, -0.950317 , -0.005538]  )

Q(   [1 , 6.646547 ,   1.520315    ]  )

Q(  [ 1, -0.300448 ,  -0.969271   ] )

Q( V_normalizada[:,2]    )